In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models
from tqdm import tqdm

In [ ]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
DEVICE = torch.device("cuda" if USE_CUDA else 'cpu')

True


In [ ]:
EPOCH = 100
Batch_Size = 64

test_acc_history = []
train_acc_history = []
test_loss_history = []
train_loss_history = []

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR100('./data',
                      train=True,
                      download=True,
                      transform = transforms.Compose([
                          transforms.RandomCrop(32, padding=4),
                          transforms.RandomHorizontalFlip(),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5),
                                               (0.5, 0.5, 0.5))
                      ])), batch_size = Batch_Size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR100('./data',
                      train=False,
                      transform = transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5),
                                               (0.5, 0.5, 0.5))
                      ])), batch_size = Batch_Size, shuffle=True
)


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 100)

    def forward(self, x):
        x = x.view(-1, 3072)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        x = self.fc3(x)

        return x

In [ ]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
def train(model, train_loader, optimizer):
    train_loss_value = 0
    train_accuracy = 0 
    running_train_loss = 0
    running_train_correct = 0

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        running_loss = 0
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output=model(data)
        
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        _,pred = torch.max(output.data, 1)
        running_train_correct += (pred == target).sum().item()
        running_train_loss += loss.item()

    train_loss_value = running_train_loss / len(train_loader)
    train_accuracy = (running_train_correct / len(train_loader.dataset) * 100)
    return train_loss_value, train_accuracy

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss=0
    correct=0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [ ]:
for epoch in tqdm(range(1, EPOCH+1)):
    train_loss, train_accuracy = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print("\n[{}] Train_loss : {:.4f}, Accuracy {:.2f}%".format(epoch, train_loss, train_accuracy))
    print('Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, test_accuracy))
        test_acc_history.append(test_accuracy)
        test_loss_history.append(test_loss)
        train_acc_history.append(train_accuracy)
        train_loss_history.append(train_loss)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

test_acc_ser = pd.Series(test_acc_history)
train_acc_ser = pd.Series(train_acc_history)
test_loss_ser = pd.Series(test_loss_history)
train_loss_ser = pd.Series(train_loss_history)

test_acc_ser.plot(label='Test')
train_acc_ser.plot(label='Train')
plt.title('DNN Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy(%)')
plt.legend(loc='best')
plt.savefig('./DNN_accuracy.png', dpi=500)
plt.show()

In [ ]:
t_l_ser = pd.Series(t_l_his)
l_ser = pd.Series(l_his)
plt.plot(l_ser, label="test")
plt.plot(t_l_ser, label="train")
plt.title('DNN Loss')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.savefig('./DNN_loss.png', dpi=500)
plt.legend(loc='best')
plt.show()

In [ ]:
torch.save(model.state_dict(), './model.pt')
print('state_dict format of the model: {}'.format(model.state_dict()))